In [1]:
from nn import NeuralNetwork, Layer, Sigmoid, Binary

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df = pd.read_csv('data.csv', index_col=0)

In [4]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y[:, np.newaxis], test_size=.33, random_state=42)

In [17]:
y_train_enc = np.where(y_train == 'B', 1, 0).astype(np.float64)
y_test_enc = np.where(y_test == 'B', 1, 0).astype(np.float64)

In [23]:
model = NeuralNetwork(loss=Binary(), n_iterations=30)
model.add(Layer(30, 1, activation=Sigmoid()))

In [19]:
X_normed = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)

In [21]:
X_test_normed = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0)

In [24]:
model.fit(X_normed, y_train_enc)

0: Binary cross-entropy=0.6951858064837787
1: Binary cross-entropy=0.6747675317956068
2: Binary cross-entropy=0.6556519732871517
3: Binary cross-entropy=0.6377478431593501
4: Binary cross-entropy=0.6209664663629236
5: Binary cross-entropy=0.6052229912670626
6: Binary cross-entropy=0.5904372255690477
7: Binary cross-entropy=0.5765341390390682
8: Binary cross-entropy=0.563444091239708
9: Binary cross-entropy=0.5511028466027125
10: Binary cross-entropy=0.5394514352899094
11: Binary cross-entropy=0.5284359099142976
12: Binary cross-entropy=0.5180070383038936
13: Binary cross-entropy=0.5081199628599398
14: Binary cross-entropy=0.4987338486361047
15: Binary cross-entropy=0.4898115353952746
16: Binary cross-entropy=0.4813192035805009
17: Binary cross-entropy=0.4732260601846548
18: Binary cross-entropy=0.4655040476731134
19: Binary cross-entropy=0.45812757715814106
20: Binary cross-entropy=0.45107328572441957
21: Binary cross-entropy=0.4443198169837239
22: Binary cross-entropy=0.43784762345362

In [32]:
accuracy_score(y_test_enc, model.predict(X_test_normed) > .5)

0.9521276595744681

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [15]:
kmodel = Sequential()
kmodel.add(Dense(30, activation='sigmoid'))
kmodel.add(Dense(1, activation='sigmoid'))
kmodel.compile('sgd', loss='binary_crossentropy', metrics=['acc'])
kmodel.fit(X_train.values, y_train_enc, epochs=70)

Epoch 1/70
380/380 [==============================] - 0s 505us/step - loss: 0.6812 - acc: 0.4763
Epoch 2/70
380/380 [==============================] - 0s 61us/step - loss: 0.6063 - acc: 0.8684
Epoch 3/70
380/380 [==============================] - 0s 61us/step - loss: 0.5802 - acc: 0.8053
Epoch 4/70
380/380 [==============================] - 0s 46us/step - loss: 0.5840 - acc: 0.8237
Epoch 5/70
380/380 [==============================] - 0s 70us/step - loss: 0.6057 - acc: 0.8132
Epoch 6/70
380/380 [==============================] - 0s 52us/step - loss: 0.6224 - acc: 0.7895
Epoch 7/70
380/380 [==============================] - 0s 72us/step - loss: 0.6108 - acc: 0.6868
Epoch 8/70
380/380 [==============================] - 0s 49us/step - loss: 0.6020 - acc: 0.6474
Epoch 9/70
380/380 [==============================] - 0s 59us/step - loss: 0.5945 - acc: 0.6500
Epoch 10/70
380/380 [==============================] - 0s 61us/step - loss: 0.5957 - acc: 0.6474
Epoch 11/70
380/380 [=================

In [22]:
kmodel = Sequential()
kmodel.add(Dense(100, activation='tanh'))
kmodel.add(Dense(100, activation='tanh'))
kmodel.add(Dense(1, activation='sigmoid'))
kmodel.compile('adam', loss='binary_crossentropy', metrics=['acc'])
kmodel.fit(X_train.values, y_train_enc, epochs=40)

Epoch 1/40
380/380 [==============================] - 1s 1ms/step - loss: 0.6636 - acc: 0.6053
Epoch 2/40
380/380 [==============================] - 0s 82us/step - loss: 0.4661 - acc: 0.8316
Epoch 3/40
380/380 [==============================] - 0s 67us/step - loss: 0.3468 - acc: 0.8974
Epoch 4/40
380/380 [==============================] - 0s 62us/step - loss: 0.3039 - acc: 0.9079
Epoch 5/40
380/380 [==============================] - 0s 87us/step - loss: 0.2832 - acc: 0.9000
Epoch 6/40
380/380 [==============================] - 0s 54us/step - loss: 0.2743 - acc: 0.9053
Epoch 7/40
380/380 [==============================] - 0s 90us/step - loss: 0.2646 - acc: 0.9079
Epoch 8/40
380/380 [==============================] - 0s 55us/step - loss: 0.2541 - acc: 0.9053
Epoch 9/40
380/380 [==============================] - 0s 58us/step - loss: 0.2525 - acc: 0.9105
Epoch 10/40
380/380 [==============================] - 0s 72us/step - loss: 0.2630 - acc: 0.9105
Epoch 11/40
380/380 [===================

In [25]:
y_test_pred = kmodel.predict_classes(X_test.values)

In [31]:
from sklearn.metrics import accuracy_score

In [27]:
accuracy_score(y_test_enc, y_test_pred)

0.9042553191489362